# prompts

## 概念

提示是赋予 LLM 表达能力的基本输入。

LlamaIndex 使用提示来

- 构建索引
- 进行插入
- 在查询期间执行遍历
- 并合成最终答案。

LlamaIndex 使用一组开箱即用的[默认提示模板](https://github.com/run-llama/llama_index/blob/main/llama-index-core/llama_index/core/prompts/default_prompts.py)。

此外，还有一些专门为聊天模型编写和使用的提示，例如gpt-3.5-turbo [这里](https://github.com/run-llama/llama_index/blob/main/llama-index-core/llama_index/core/prompts/chat_prompts.py)。

## 使用模式

In [ ]:
from llama_index.core import PromptTemplate

template = (
    "We have provided context information below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Given this information, please answer the question: {query_str}\n"
)
qa_template = PromptTemplate(template)

# you can create text prompt (for completion API)
prompt = qa_template.format(context_str=..., query_str=...)

# or easily convert to message prompts (for chat API)
messages = qa_template.format_messages(context_str=..., query_str=...)

从聊天消息中定义模板

In [ ]:
from llama_index.core import ChatPromptTemplate
from llama_index.core.llms import ChatMessage, MessageRole

message_templates = [
    ChatMessage(content="You are an expert system.", role=MessageRole.SYSTEM),
    ChatMessage(
        content="Generate a short story about {topic}",
        role=MessageRole.USER,
    ),
]
chat_template = ChatPromptTemplate(message_templates=message_templates)

# you can create message prompts (for chat API)
messages = chat_template.format_messages(topic=...)

# or easily convert to text prompt (for completion API)
prompt = chat_template.format(topic=...)

## 常用提示

最常用的提示是text_qa_template和refine_template。

- text_qa_template- 用于使用检索到的节点获取查询的初始答案
- refine_template- 当检索到的文本不适合单个 LLM 调用response_mode="compact"（默认）时使用，或者当使用检索多个节点时使用response_mode="refine"。第一个查询的答案作为插入existing_answer，LLM 必须根据新上下文更新或重复现有答案。

## 查看当前环境的提示

In [ ]:
query_engine = index.as_query_engine(response_mode="compact")
prompts_dict = query_engine.get_prompts()
print(list(prompts_dict.keys()))

## 更新提示

In [ ]:
# shakespeare!
qa_prompt_tmpl_str = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the query in the style of a Shakespeare play.\n"
    "Query: {query_str}\n"
    "Answer: "
)
qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)

query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)

## 修改查询引擎中使用的提示

In [ ]:
# 高级 API
query_engine = index.as_query_engine(
    text_qa_template=custom_qa_prompt, refine_template=custom_refine_prompt
)

# 底层 api
retriever = index.as_retriever()
synth = get_response_synthesizer(
    text_qa_template=custom_qa_prompt, refine_template=custom_refine_prompt
)
query_engine = RetrieverQueryEngine(retriever, response_synthesizer)

## 修改索引构建中使用的提示

一些索引在构建过程中使用不同类型的提示（注意：最常见的VectorStoreIndex和SummaryIndex不使用任何提示）。

In [ ]:
# TreeIndex
index = TreeIndex(nodes, summary_template=custom_prompt)

# 另一种方式
index = TreeIndex.from_documents(docs, summary_template=custom_prompt)

## 高级提示功能

### 部分格式化

部分格式化提示，填写一些变量，同时将其他变量留待以后填写。

### 模板变量映射

将当前环境的变量映射为模版的变量。

### 函数映射

传入函数作为模板变量而不是固定值。